In [326]:
import numpy as np
import pandas as pd
import sympy as sp
import unittest
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [327]:
class Laguerre:
    def __init__(self, T, N, beta, sigma, epsilon, t_values):
        self.T = T
        self.N = N
        self.beta = beta
        self.sigma = sigma
        self.epsilon = epsilon
        self.t_values = t_values
        self.alpha = self.sigma - self.beta
    
    
    @property
    def T(self):
        return self._T
    @T.setter
    def T(self, T):
        self._T = T
        
    @property
    def N(self):
        return self._N
    @N.setter
    def N(self, N):
        self._N = N
               
    @property
    def beta(self):
        return self._beta
    @beta.setter
    def beta(self, beta):
        self._beta = beta
        
    @property
    def sigma(self):
        return self._sigma
    @sigma.setter
    def sigma(self, sigma):
        self._sigma = sigma        
        
    @property
    def epsilon(self):
        return self._epsilon
    @epsilon.setter
    def epsilon(self, epsilon):
        self._epsilon = epsilon
          
    @property
    def t_values(self):
        return self._t_values
    @t_values.setter
    def t_values(self, t_values):
        self._t_values = t_values
        
    def laguerre(self, n, t):
        if (self.beta>=0) and (self.beta<=self.sigma):
            if n==0:
                return np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
            elif n==1:
                return np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
            else:
                l0=np.sqrt(self.sigma)*np.exp((-self.beta*t)/2)
                l1=np.sqrt(self.sigma)*(1-self.sigma*t)*np.exp((-self.beta*t)/2)
                for i in range(2, n+1):
                    li=((2*i-1-self.sigma*t)/i)*l1 - ((i-1)/i)*l0
                    l0, l1 = l1, li
                return li

        else:
            raise ValueError("Wrong data input! Beta should be in range [0;sigma]!")
        
    
    def tabulate_laguerre(self): 
        laguerre_val = {"t_val": self.t_values}  
        for n in range(self.N + 1):
            laguerre_val[f"L_{n}"] = [self.laguerre(n, t) for t in self.t_values]
        
        return pd.DataFrame(laguerre_val)
    
        
    def find_t_epsilon(self, n, epsilon=1e-3, max_T=100, steps=1000):
        t_values = np.linspace(0, max_T, steps)
        for t in t_values:
            if all(np.abs(self.laguerre(i, t)) < epsilon for i in range(n)):
                return t
        raise ValueError("No value t")

In [328]:
class Integrate(Laguerre):
    def __init__(self, laguerre_object, max_steps=10000):
        super().__init__(laguerre_object.T, laguerre_object.N, laguerre_object.beta, laguerre_object.sigma, laguerre_object.epsilon, laguerre_object.t_values)
        self.max_steps = max_steps

    def integrate_laguerre(self, f, k):
        alpha = self.sigma - self.beta
        steps = 10
        prev_integral = 0
        delta = float("inf")
    
        while delta > self.epsilon:
            t_values = np.linspace(0, self.T, steps + 1)
            delta_t = self.T / steps
        
            # Ось тут виправлено
            laguerre_values = np.array([self.laguerre(k, t) for t in t_values])
            integrand = f(t_values) * laguerre_values * np.exp(-alpha * t_values)
            integral = np.sum(integrand * delta_t) 
        
            delta = abs(integral - prev_integral)
            prev_integral = integral
        
            steps *= 2
            if steps > self.max_steps:
                raise RuntimeError("Максимальна кількість кроків перевищена. Інтеграл не збігається.")
    
        return prev_integral


In [329]:
class ReverseLaguerre(Laguerre):
    def __init__(self, laguerre_object, coefficients=None):
        super().__init__(laguerre_object.T, laguerre_object.N, laguerre_object.beta, laguerre_object.sigma, laguerre_object.epsilon, laguerre_object.t_values)
        self.coefficients = coefficients if coefficients is not None else []

    @property
    def coefficients(self):
        return self._coefficients

    @coefficients.setter
    def coefficients(self, coefficients):
        self._coefficients = coefficients

    def laguerre_coefficients(self, integr_obj, f):
        coeffs = []
        for k in range(self.N + 1): 
            coeff = integr_obj.integrate_laguerre(f, k)
            coeffs.append(coeff)
        self.coefficients = coeffs 
        return coeffs 

    def reverse_laguerre(self, t):
        if len(self.coefficients) == 0:
            raise ValueError("Coefficients are not computed")
        
        result = 0
        for k in range(len(self.coefficients)):
            result += self.coefficients[k] * self.laguerre(k, t)
        return result

    def conditional_func(self, t_values=None):
        if t_values is None:
            t_values = np.linspace(0, self.T, 100)
        return np.array([self.reverse_laguerre(t) for t in t_values])


In [330]:
class TestLaguerre(unittest.TestCase):
	def setUp(self):
		self.laguerre_object = Laguerre(10,20,-1,40,10e3,[1,2,3,4])
		self.integral = Integrate(self.laguerre_object, 0)
		self.reversed_values = ReverseLaguerre(self.laguerre_object, [])

	def test_laguerre_main_function(self):
		with self.assertRaises(ValueError):
			"""Test with incorrect beta"""
			self.laguerre_object.laguerre(1,3)

	def test_laguerre_rec_function(self):
		"""Not callable function (string type)"""
		self.laguerre_object.beta = 10
		with self.assertRaises(ValueError):
			self.integral.rect_integrate_laguerre(10, "np.sin")

	def test_laguerre_trapez_function(self):
		"""Not callable function (string type)"""
		with self.assertRaises(ValueError):
			self.integral.rect_integrate_laguerre(10, "np.sin")

	def test_laguerre_revers_coffs(self):
		"""Empty list of coefficients"""
		with self.assertRaises(ValueError):
			self.reversed_values.reverse_laguerre(self.integral, 10)

	def test_is_equel_integrals(self):
		val1 = self.integral.rect_integrate_laguerre(10, np.sin)
		val2 = self.integral.trapez_integrate_laguerre(10, np.sin)
		self.assertEqual(val1, val2)


In [331]:
#unittest.main(argv=[''], exit=False)

In [332]:
class Visualization:
    @staticmethod
    def plot_laguerre(laguerre_obj, n, t):
        t_values = np.linspace(0, t, 100)
        y_values = [laguerre_obj.laguerre(n, t_i) for t_i in t_values]
        
        plt.figure(figsize=(8, 5))
        plt.plot(t_values, y_values, label=f'L_{n}(t)')
        plt.xlabel('t')
        plt.ylabel(f'L_{n}(t)')
        plt.title('Laguerre Polynomial')
        plt.legend()
        plt.grid()
        plt.show()
    
    @staticmethod
    def tabulate_laguere(laguerre_obj, n, T, steps=100):
        t_values = np.linspace(0, T, steps)
        results = {f'L_{i}': [laguerre_obj.laguerre(i, t) for t in t_values] for i in range(n)}
        results['t'] = t_values
        df = pd.DataFrame(results)
        display(df)
    
    @staticmethod
    def epsilon_vis(laguerre_obj, n):
        return laguerre_obj.find_t_epsilon(n)

    @staticmethod
    def integrate_vis(integrate_obj, n, f):
        return integrate_obj.integrate_laguerre(f, n)

    @staticmethod
    def coef_vis(reverse_obj, integrate_obj, f):
        coefficients = reverse_obj.laguerre_coefficients(integrate_obj, f)
        print("Coefficients:", coefficients)
        return coefficients
    
    @staticmethod
    def reverse_plot(reverse_obj, t_values):
        if len(reverse_obj.coefficients) == 0:
            raise ValueError("Coefficients are not computed")
        
        y_values = [reverse_obj.reverse_laguerre(t) for t in t_values]
        
        plt.figure(figsize=(8, 5))
        plt.plot(t_values, y_values, label='Reverse Laguerre Transform')
        plt.xlabel('t')
        plt.ylabel('f(t)')
        plt.title('Reverse Laguerre Transform')
        plt.legend()
        plt.grid()
        plt.show()
    
    @staticmethod
    def output(laguerre_obj, integrate_obj, reverse_obj):
        dropdown = widgets.Dropdown(
            options=['plot_laguerre', 'tabulate_laguere', 'epsilon_vis', 'integrate_vis', 'coef_vis', 'reverse_plot'],
            description='Виберіть дію:',
        )

        output_area = widgets.Output()
        f_input = widgets.Text(value="np.sin", description="Функція:")
        n_slider = widgets.IntSlider(value=3, min=1, max=10, description="n:")
        t_slider = widgets.FloatSlider(value=5, min=0, max=20, description="t:")

        def update_plot(change=None):
            output_area.clear_output()
            f = eval(f_input.value)
            with output_area:
                if dropdown.value == "plot_laguerre":
                    Visualization.plot_laguerre(laguerre_obj, n_slider.value, t_slider.value)
                elif dropdown.value == "tabulate_laguere":
                    Visualization.tabulate_laguere(laguerre_obj, n_slider.value, t_slider.value)
                elif dropdown.value == "epsilon_vis":
                    print(Visualization.epsilon_vis(laguerre_obj, n_slider.value))
                elif dropdown.value == "integrate_vis":
                    print(Visualization.integrate_vis(integrate_obj, n_slider.value, f))
                elif dropdown.value == "coef_vis":
                    Visualization.coef_vis(reverse_obj, integrate_obj, f)
                elif dropdown.value == "reverse_plot":
                    t_values = np.linspace(0, t_slider.value, 100)
                    Visualization.reverse_plot(reverse_obj, t_values)
                else:
                    print("Choose option!")

        dropdown.observe(update_plot, names='value')
        n_slider.observe(update_plot, names='value')
        t_slider.observe(update_plot, names='value')
        f_input.observe(update_plot, names='value')

        display(f_input, n_slider, t_slider, dropdown, output_area)

In [333]:
laguerre_object = Laguerre(10, 20, 2, 40, 10e3, [110, 220, 350, 13900])
integral = Integrate(laguerre_object)
reversed_values = ReverseLaguerre(laguerre_object, [])

In [334]:
Visualization.output(laguerre_object,integral,reversed_values)

Text(value='np.sin', description='Функція:')

IntSlider(value=3, description='n:', max=10, min=1)

FloatSlider(value=5.0, description='t:', max=20.0)

Dropdown(description='Виберіть дію:', options=('plot_laguerre', 'tabulate_laguere', 'epsilon_vis', 'integrate_…

Output()